In [1]:
import pyBigWig
import csv
import pandas as pd
import numpy as np

print("Completed importing!")

Completed importing!


In [7]:
############################################
# Calculate BPM for each specified region in bigwig files
############################################

# Function to read BPM values from a BigWig file
def read_bpm_from_bigwig(bigwig_file, chrom, start, stop):
    with pyBigWig.open(bigwig_file) as bw:
        values = bw.values(chrom, start, stop)
        # Handle NaN values or processing here as needed
        return sum(values) / len(values) if values else 0

# Path to your TSV file
tsv_file = '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/DiffPeaks/OPN5vVEH_genes_filtered_fromPeaks_forAudrey.tsv'

# List of BigWig files
bigwig_files = ['/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/DiffPeaks/2160_PLX5.pval.signal.bigwig', 
                '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/DiffPeaks/2160_VEH.pval.signal.bigwig',
                '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/DiffPeaks/2161_PLX5.pval.signal.bigwig', 
                '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/DiffPeaks/2161_VEH.pval.signal.bigwig',
                '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/DiffPeaks/2281_PLX5.pval.signal.bigwig',
                '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/DiffPeaks/2281_VEH.pval.signal.bigwig',
                '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/DiffPeaks/2920_PLX5.pval.signal.bigwig',
                '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/DiffPeaks/2920_VEH.pval.signal.bigwig']

# Read TSV file and process each line
with open(tsv_file, 'r') as infile, open('/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/DiffPeaks/OPN5vVEH_genes_with_ATAC_signal.tsv', 'w', newline='') as outfile:
    reader = csv.reader(infile, delimiter='\t')
    writer = csv.writer(outfile, delimiter='\t')

    # Write the header
    header = ['Chr', 'Start', 'End', 'Gene_Name'] + ['Sample_' + str(i+1) for i in range(len(bigwig_files))]
    writer.writerow(header)

    # Process each line of the TSV file
    for row in reader:
        chrom, start, stop, gene = row
        start, stop = int(start), int(stop)

        # Extract BPM values for each sample
        bpm_values = [read_bpm_from_bigwig(bw_file, chrom, start, stop) for bw_file in bigwig_files]

        # Write the updated row to the output file
        writer.writerow(row + bpm_values)

print(f"Processing complete.")


Processing complete. Output saved to 'Master_gene_signal.tsv'.


In [46]:
'''
Do you want the average signal across samples or individual comparisons by sample name?
Calculate average signal here, sample by sample below.
'''

def calculate_average_signals(file_path, veh_columns, opn5_columns):
    """
    Calculate average signals for specified groups of columns.

    :param file_path: Path to the file with the data.
    :param veh_columns: List of column names for VEH samples.
    :param opn5_columns: List of column names for OPN5 samples.
    """
    # Load the data
    data = pd.read_csv(file_path, delimiter='\t')

    # Calculate average signals for the specified groups
    data['avg_VEH'] = data[veh_columns].median(axis=1)
    data['avg_OPN5'] = data[opn5_columns].median(axis=1)

    # Create a new DataFrame with required columns
    new_data = data[['chromosome', 'start', 'stop', 'gene', 'avg_VEH', 'avg_OPN5']]

    # Save the new DataFrame to a file
    output_file = '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/GenomeAnnotation/Master_genes_median_signals.tsv'
    new_data.to_csv(output_file, sep='\t', index=False)
    print(f"Average signal values written to '{output_file}'.")

# Path to your merged peaks file
file_path = '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/GenomeAnnotation/Master_genes_signal.tsv'

# Specify the columns for VEH and OPN5 samples
veh_columns = ['VEH_2160', 'VEH_2161', 'VEH_2281', 'VEH_2920']  # Replace with actual VEH column names
opn5_columns = ['OPN5_2160', 'OPN5_2161', 'OPN5_2281', 'OPN5_2920']  # Replace with actual OPN5 column names

calculate_average_signals(file_path, veh_columns, opn5_columns)



Average signal values written to '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/GenomeAnnotation/Master_genes_median_signals.tsv'.


In [47]:
############################################
# Calculate fold changes
############################################

# Load the data
file_path = '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/GenomeAnnotation/Master_genes_median_signals.tsv'
data = pd.read_csv(file_path, delimiter='\t')

# Specify the column pairs for Vehicle and OPN5 treatments
# Update these with the actual column names
vehicle_columns = ['avg_VEH']#['VEH_2160', 'VEH_2161', 'VEH_2281', 'VEH_2920']
opn5_columns = ['avg_OPN5']#['OPN5_2160', 'OPN5_2161', 'OPN5_2281', 'OPN5_2920']

# Ensure that the number of vehicle columns matches the number of OPN5 columns
if len(vehicle_columns) != len(opn5_columns):
    raise ValueError("The number of Vehicle and OPN5 samples must be the same.")

# Calculate fold change for each pair (o/v)
fold_changes = pd.DataFrame()
for v_col, o_col in zip(vehicle_columns, opn5_columns):
    fold_change_col = f"Fold_Change_{v_col}_vs_{o_col}"
    fold_changes[fold_change_col] = data[o_col] / data[v_col]
    
# Calculate fold change for each pair (v/o)
# fold_changes = pd.DataFrame()
# for v_col, o_col in zip(vehicle_columns, opn5_columns):
#     fold_change_col = f"Fold_Change_{o_col}_vs_{v_col}"
#     fold_changes[fold_change_col] = data[v_col] / data[o_col]

# Concatenate the original data with the fold change results
result = pd.concat([data[['chromosome', 'start', 'stop', 'gene']], fold_changes], axis=1)

# Save the result to a new TSV file
output_file = '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/GenomeAnnotation/Master_median_fold_change.tsv'
result.to_csv(output_file, sep='\t', index=False)

print(f"Paired fold change calculation complete. Results saved to '{output_file}'.")


Paired fold change calculation complete. Results saved to '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/GenomeAnnotation/Master_median_fold_change.tsv'.


In [28]:
############################################
# Filter genes based on fold change for GO analysis
############################################

def filter_genes(file_path, cutoff, min_samples=4):
    """
    Filter genes based on fold change values and output to a text file.

    :param file_path: Path to the TSV file with fold change data.
    :param cutoff: The cutoff value for fold change.
    :param min_samples: Minimum number of samples that should meet the cutoff (default 4).
    :return: None. Outputs results to a text file.
    """
    # Load the data
    data = pd.read_csv(file_path, delimiter='\t')

    # Select only fold change columns
    fold_change_cols = [col for col in data.columns if 'Fold_Change' in col]

    # Apply the cutoff and count the number of times each gene meets the criteria
    data['above_cutoff'] = data[fold_change_cols].ge(cutoff).sum(axis=1)

    # Filter genes that meet the criteria in the specified number of samples
    filtered_genes = data[data['above_cutoff'] >= min_samples]['gene']

    print(str(filtered_genes))
    
    # Remove duplicates and sort
    unique_genes = sorted(set(gene for gene in filtered_genes if isinstance(gene, str)))

    # Write to a text file
    with open(f'filtered_genes_min_{min_samples}_cutoff_{cutoff}.txt', 'w') as file:
        for gene in unique_genes:
            file.write(f'{gene}\n')

    print(f"Filtered gene list written to 'filtered_genes_min_{min_samples}_cutoff_{cutoff}.txt'")

# File path to the TSV file
file_path = '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/GenomeAnnotation/VEH_paired_fold_change_output.tsv'

# Set your cutoff value here
cutoff = 2.0  # Replace YOUR_CUTOFF_VALUE with your chosen cutoff value

# Output genes meeting the cutoff in all four samples to a file
filter_genes(file_path, cutoff, min_samples=4)

# Output genes meeting the cutoff in at least three samples to a file
filter_genes(file_path, cutoff, min_samples=3)

5             ADAM12
14             ITGB1
15             ITGB1
16             ITGB1
17             ITGB1
            ...     
1196             VCP
1202    LOC105376114
1217          CD40LG
1218          CD40LG
1235         ARHGEF9
Name: gene, Length: 183, dtype: object
Filtered gene list written to 'filtered_genes_min_4_cutoff_2.0.txt'
0       OLMALINC
4         ADAM12
5         ADAM12
14         ITGB1
15         ITGB1
          ...   
1235     ARHGEF9
1237    FLJ44635
1238    FLJ44635
1239    FLJ44635
1240     MIR374B
Name: gene, Length: 631, dtype: object
Filtered gene list written to 'filtered_genes_min_3_cutoff_2.0.txt'


In [ ]:
###
# Now starts the calculations for GSEA
###

In [ ]:
# Find average fold change across all patients at significant Manorm peak list.
# Master peak list -> BPM values of all samples at all those sites -> calculate average BPM across VEH and OPN5 
# -> average FC calculation -> geom mean the average FC for RNK file, submit to easyGSEA

# Currently have Master gene lists biased towards VEH and OPN5, but I need the unbiased peaks as well...
# Commands to convert peak list to gene positions:

# Grab the necessary columns if the row contains a promoter proximal peak.
!awk -F'\t' '$8 ~ /promoter|TSS/ {print $2"\t"$3"\t"$4"\t"$16}' Patient_2160/Unbiased_Annotated.txt > Patient_2160/Unbiased_genes_positions.txt

# Just give me the unique ones, there should be no duplicate lines
!cat Patient_2160/Unbiased_genes_positions.txt | sort | uniq > Patient_2160/Unbiased_genes_positions_nodups.txt

# Join them all together and remove duplicates
!cat Patient_2160/Unbiased_genes_positions_nodups.txt Patient_2161/Unbiased_genes_positions_nodups.txt Patient_2281/Unbiased_genes_positions_nodups.txt Patient_2920/Unbiased_genes_positions_nodups.txt | sort | uniq > Unbiased_Master_genes_positions.txt

# To fix multiple peaks being in the same promoter region, find overlap between peaks and concatanate them.
# So if the region described by the first peak overlaps the next peak, make the end position of the next 
# peak the end position of this peak and delete that row.


In [18]:
############################################
# Ensure all overlapping peaks are merged together to reduce overlap
############################################
def merge_overlapping_peaks(file_path):
    # Load the data
    data = pd.read_csv(file_path, delimiter='\t', names=['chromosome', 'start', 'end', 'gene_name'])

    # Sort data by chromosome and start position
    data.sort_values(by=['chromosome', 'gene_name', 'start'], inplace=True)

    # Initialize a new DataFrame for the merged peaks
    merged_data = pd.DataFrame(columns=['chromosome', 'start', 'end', 'gene_name'])
    current_row = None

    # Iterate through the rows
    for index, row in data.iterrows():
        if current_row is not None and row['gene_name'] == current_row['gene_name']:
            # Check for overlap and merge if necessary
            if row['start'] <= current_row['end']:
                current_row['end'] = max(current_row['end'], row['end'])
                continue
        # Add the previous row to the merged data
        if current_row is not None:
            merged_data = merged_data.append(current_row, ignore_index=True)
        current_row = row

    # Add the last row
    merged_data = merged_data.append(current_row, ignore_index=True)

    # Save the merged data to a new TSV file
    output_file = '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/GenomeAnnotation/Master_gene_positions_merged_peaks.tsv'
    merged_data.to_csv(output_file, sep='\t', index=False, header=False)
    print(f"Merged peaks written to '{output_file}'.")

# Path to your TSV file
file_path = '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/GenomeAnnotation/Tex_OCRs_hg38/master_OCR_signal.bed'
merge_overlapping_peaks(file_path)


Merged peaks written to '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/GenomeAnnotation/Tex_OCRs_hg38/combined_output_merged_peaks.bed'.


In [21]:
############################################
# Combine overlap on next line
############################################
def merge_overlapping_peaks(file_path):
    # Load the data
    data = pd.read_csv(file_path, delimiter='\t', names=['chromosome', 'start', 'end'], usecols=[0, 1, 2])

    # Sort data by chromosome and start position just in case
    data.sort_values(by=['chromosome', 'start'], inplace=True)

    # Initialize a new DataFrame for the merged peaks
    merged_data = pd.DataFrame(columns=['chromosome', 'start', 'end'])
    current_chrom = None
    current_start = None
    current_end = None

    # Iterate through the rows
    for index, row in data.iterrows():
        # If this is the first row or a new chromosome, reset the current variables
        if current_chrom is None or row['chromosome'] != current_chrom:
            # Add the previous row to the merged data if it exists
            if current_chrom is not None:
                merged_data = merged_data.append({'chromosome': current_chrom, 'start': current_start, 'end': current_end}, ignore_index=True)
            current_chrom = row['chromosome']
            current_start = row['start']
            current_end = row['end']
        else:
            # If the current row overlaps with the previous, merge them
            if row['start'] <= current_end:
                current_end = max(current_end, row['end'])
            else:
                # If it doesn't overlap, add the previous range and start a new one
                merged_data = merged_data.append({'chromosome': current_chrom, 'start': current_start, 'end': current_end}, ignore_index=True)
                current_start = row['start']
                current_end = row['end']

    # Add the last row
    merged_data = merged_data.append({'chromosome': current_chrom, 'start': current_start, 'end': current_end}, ignore_index=True)

    # Save the merged data to a new TSV file
    output_file = '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/DiffPeaks/merged_VEH_peaks.bed'
    merged_data.to_csv(output_file, sep='\t', index=False, header=False)
    print(f"Merged peaks written to '{output_file}'.")

# Path to your TSV file
file_path = '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/DiffPeaks/uniq_combined_VEH_peaks.bed'
merge_overlapping_peaks(file_path)

Merged peaks written to '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/DiffPeaks/merged_VEH_peaks.bed'.


In [ ]:
# Now with this, return to the start and get the BPM values for these genes.

In [48]:
############################################
# For GSEA, we need the fold change across the entire gene, so get the median of fold change values
############################################
def condense_genes(file_path):
    """
    Condense each gene into a single row by calculating the median fold change value.

    :param file_path: Path to the file with gene data and fold change values.
    """
    # Load the data
    data = pd.read_csv(file_path, delimiter='\t')

    # Calculate the median fold change for each gene
    median_data = data.groupby('gene').median()

    # Reset index to turn the gene names back into a column
    median_data.reset_index(inplace=True)

    # Select relevant columns if necessary
    # If you want to include other columns like chromosome, start, and end, you might need additional processing
    median_data = median_data[['gene', 'Fold_Change_avg_VEH_vs_avg_OPN5']]

    # Save the result to a new file
    output_file = '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/GenomeAnnotation/Master_median_fold_change_condensed.tsv'
    median_data.to_csv(output_file, sep='\t', index=False)
    print(f"Median fold change for each gene is written to '{output_file}'.")

# Path to your file containing the gene data
file_path = '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/GenomeAnnotation/Master_median_fold_change.tsv'
condense_genes(file_path)

Median fold change for each gene is written to '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/GenomeAnnotation/Master_median_fold_change_condensed.tsv'.


In [ ]:
# Combine unbiased, OPN5 and VEH signal files...................run them through the process

In [60]:
############################################
# Make RNK file from fold change output
############################################
def create_rnk_file(file_path, output_file):
    """
    Convert fold change data into a .rnk file for GSEA analysis.

    :param file_path: Path to the file with fold change data.
    :param output_file: Path to the output .rnk file.
    """
    # Load the data
    data = pd.read_csv(file_path, delimiter='\t')

    # Print out the column names to check
    print("Column names in the file:", data.columns.tolist())

    # Assuming the fold change column is named 'change'. Adjust if it has a different name.
    fold_change_column = 'change'

    # Ensure the column exists
    if fold_change_column not in data.columns:
        raise ValueError(f"Column '{fold_change_column}' not found in the file.")

    # Transform fold change: log2 transformation
    data['transformed_fc'] = np.log2(data[fold_change_column].replace(0, np.nan))

    # Drop NaN values (originally 1s in fold change)
    data.dropna(subset=['transformed_fc'], inplace=True)

    # Rank genes
    ranked_genes = data[['gene', 'transformed_fc']].sort_values(by='transformed_fc', ascending=False)

    # Save to .rnk file
    ranked_genes.to_csv(output_file, sep='\t', index=False, header=False)
    print(f"RNK file created: '{output_file}'.")

# Paths to your input and output files
file_path = '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/GenomeAnnotation/Master_median_fold_change_condensed.tsv'
output_rnk_file = '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/GenomeAnnotation/Master_ATAC_genes.rnk'
create_rnk_file(file_path, output_rnk_file)


Column names in the file: ['gene', 'change']
RNK file created: '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/GenomeAnnotation/Master_ATAC_genes.rnk'.


In [2]:
############################################
# Keep only one gene by averaging sample signal between sites.
############################################

def process_duplicates(input_file, output_file):
    # Read in the TSV file
    df = pd.read_csv(input_file, sep='\t')

    # Group by 'gene' and calculate the mean for each sample column
    sample_columns = ['OPN5_2160', 'VEH_2160', 'OPN5_2161', 'VEH_2161', 'OPN5_2281', 'VEH_2281', 'OPN5_2920', 'VEH_2920']
    grouped_df = df.groupby('gene')[sample_columns].mean().reset_index()

    # Merge the grouped data back with the original dataframe to get the non-sample columns
    merged_df = df.drop(columns=sample_columns).drop_duplicates('gene').merge(grouped_df, on='gene')

    # Write the results to a new TSV file
    merged_df.to_csv(output_file, sep='\t', index=False)

# Usage
input_file = '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/GenomeAnnotation/Master_genes_signal.tsv'
output_file = '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/GenomeAnnotation/Master_genes_signal_condensed.tsv'
process_duplicates(input_file, output_file)

In [9]:
############################################
# Filling in some gaps:
############################################

bigwig_files = ['/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/atac/a713b253-c16b-4125-ba5b-4ddbe1de071f/call-macs2_signal_track/shard-0/execution/2160_PLX5_S2_L001_R1_001.merged.srt.nodup.no_chrM_MT.tn5.pval.signal.bigwig', 
                '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/atac/a713b253-c16b-4125-ba5b-4ddbe1de071f/call-macs2_signal_track/shard-1/execution/2160_VEH_S1_L001_R1_001.trim.merged.srt.nodup.no_chrM_MT.tn5.pval.signal.bigwig', 
                '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/atac/a713b253-c16b-4125-ba5b-4ddbe1de071f/call-macs2_signal_track/shard-2/execution/2161_PLX5_S8_L001_R1_001.trim.merged.srt.nodup.no_chrM_MT.tn5.pval.signal.bigwig',
                '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/atac/a713b253-c16b-4125-ba5b-4ddbe1de071f/call-macs2_signal_track/shard-3/execution/2161_VEH_S7_L001_R1_001.trim.merged.srt.nodup.no_chrM_MT.tn5.pval.signal.bigwig',
                '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/atac/a713b253-c16b-4125-ba5b-4ddbe1de071f/call-macs2_signal_track/shard-4/execution/2281_PLX5_S4_L001_R1_001.trim.merged.srt.nodup.no_chrM_MT.tn5.pval.signal.bigwig',
                '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/atac/a713b253-c16b-4125-ba5b-4ddbe1de071f/call-macs2_signal_track/shard-5/execution/2281_VEHS3_L001_R1_001.trim.merged.srt.nodup.no_chrM_MT.tn5.pval.signal.bigwig',
                '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/atac/a713b253-c16b-4125-ba5b-4ddbe1de071f/call-macs2_signal_track/shard-6/execution/2920_PLX5_S6_L001_R1_001.trim.merged.srt.nodup.no_chrM_MT.tn5.pval.signal.bigwig',
                '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/atac/a713b253-c16b-4125-ba5b-4ddbe1de071f/call-macs2_signal_track/shard-7/execution/2920_VEH_S5_L001_R1_001.trim.merged.srt.nodup.no_chrM_MT.tn5.pval.signal.bigwig']

# Function to read BPM values from a BigWig file
def read_bpm_from_bigwig(bigwig_file, chrom, start, stop):
    with pyBigWig.open(bigwig_file) as bw:
        values = bw.values(chrom, start, stop)
        # Handle NaN values or processing here as needed
        return sum(values) / len(values) if values else 0


    
chrom = "chr1"
start = 101236558
stop = 101236977

# Process each line of the TSV file
for file in bigwig_files:

    print(f"Dumping BPM for {chrom} {start}-{stop}")
    # Extract BPM values for each sample
    bpm_values = read_bpm_from_bigwig(file, chrom, start, stop)
    print(f"{file} signal at this locus is: {bpm_values}")

Dumping BPM for chr1 101236558-101236977
/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/atac/a713b253-c16b-4125-ba5b-4ddbe1de071f/call-macs2_signal_track/shard-0/execution/2160_PLX5_S2_L001_R1_001.merged.srt.nodup.no_chrM_MT.tn5.pval.signal.bigwig signal at this locus is: 49.70158002200161
Dumping BPM for chr1 101236558-101236977
/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/atac/a713b253-c16b-4125-ba5b-4ddbe1de071f/call-macs2_signal_track/shard-1/execution/2160_VEH_S1_L001_R1_001.trim.merged.srt.nodup.no_chrM_MT.tn5.pval.signal.bigwig signal at this locus is: 31.45534738112321
Dumping BPM for chr1 101236558-101236977
/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/atac/a713b253-c16b-4125-ba5b-4ddbe1de071f/call-macs2_signal_track/shard-2/execution/2161_PLX5_S8_L001_R1_001.trim.merged.srt.nodup.no_chrM_MT.tn5.pval.signal.bigwig signal at this locus is: 63.15049863032339
Dumping BPM for chr1 101236558-101236977
/Zulu/timrez/Projects/ElGamal_D/TCell_ATA

In [10]:
bigwig_files = ['/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/atac/a713b253-c16b-4125-ba5b-4ddbe1de071f/call-macs2_signal_track/shard-0/execution/2160_PLX5_S2_L001_R1_001.merged.srt.nodup.no_chrM_MT.tn5.pval.signal.bigwig', 
                '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/atac/a713b253-c16b-4125-ba5b-4ddbe1de071f/call-macs2_signal_track/shard-1/execution/2160_VEH_S1_L001_R1_001.trim.merged.srt.nodup.no_chrM_MT.tn5.pval.signal.bigwig', 
                '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/atac/a713b253-c16b-4125-ba5b-4ddbe1de071f/call-macs2_signal_track/shard-2/execution/2161_PLX5_S8_L001_R1_001.trim.merged.srt.nodup.no_chrM_MT.tn5.pval.signal.bigwig',
                '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/atac/a713b253-c16b-4125-ba5b-4ddbe1de071f/call-macs2_signal_track/shard-3/execution/2161_VEH_S7_L001_R1_001.trim.merged.srt.nodup.no_chrM_MT.tn5.pval.signal.bigwig',
                '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/atac/a713b253-c16b-4125-ba5b-4ddbe1de071f/call-macs2_signal_track/shard-4/execution/2281_PLX5_S4_L001_R1_001.trim.merged.srt.nodup.no_chrM_MT.tn5.pval.signal.bigwig',
                '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/atac/a713b253-c16b-4125-ba5b-4ddbe1de071f/call-macs2_signal_track/shard-5/execution/2281_VEHS3_L001_R1_001.trim.merged.srt.nodup.no_chrM_MT.tn5.pval.signal.bigwig',
                '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/atac/a713b253-c16b-4125-ba5b-4ddbe1de071f/call-macs2_signal_track/shard-6/execution/2920_PLX5_S6_L001_R1_001.trim.merged.srt.nodup.no_chrM_MT.tn5.pval.signal.bigwig',
                '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/atac/a713b253-c16b-4125-ba5b-4ddbe1de071f/call-macs2_signal_track/shard-7/execution/2920_VEH_S5_L001_R1_001.trim.merged.srt.nodup.no_chrM_MT.tn5.pval.signal.bigwig']


# (gene_name, chrom, start, stop)
regions = [
    ('CD8A', 'chr2', 86790848, 86791100),
    ('CCL5', 'chr17', 35880239, 35880544),
    ('CCL4', 'chr17', 36103374, 36103986),
    ('CD70', 'chr19', 6590854, 6591117),
    ('PLSCR1', 'chr3', 146544453, 146544820),
    ('FAM3C', 'chr7', 121396300, 121396612),
    ('FAM3C', 'chr7', 121396300, 121396612),
]

# Function to read average signal from a BigWig file for given coordinates
def read_avg_signal_from_bigwig(bigwig_file, chrom, start, stop):
    with pyBigWig.open(bigwig_file) as bw:
        values = bw.values(chrom, start, stop)
        # Compute average; handle cases where values list is empty
        return sum(values) / len(values) if values else 0

# Process each region and write the results to a TSV file
with open('/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/GenomeAnnotation/Master_Extended_GSVA_Matrix.tsv', 'w', newline='') as tsvfile:
    tsvwriter = csv.writer(tsvfile, delimiter='\t')
    # Write header row with file names as column titles
    header = ['Gene Name'] + [f'File{i+1}' for i in range(len(bigwig_files))]
    tsvwriter.writerow(header)
    
    for gene_name, chrom, start, stop in regions:
        # Initialize row with gene name
        row = [gene_name]
        # Append average signal from each BigWig file
        for file in bigwig_files:
            avg_signal = read_avg_signal_from_bigwig(file, chrom, start, stop)
            row.append(avg_signal)
        # Write the row to the TSV file
        tsvwriter.writerow(row)

print("Done writing to output.tsv")

Done writing to output.tsv


In [13]:
########################
# Combine MAnorm file with annotations.
########################

# Load the original combined MAnorm file
manorm_df = pd.read_csv('/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/DiffPeaks/Patient_2920/OPN5_vs_VEH_mavalues.tsv', sep='\t')

# Load the HOMER annotated file
annotated_df = pd.read_csv('/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/DiffPeaks/Patient_2920/OPN5_vs_VEH_annotatedPeaks.txt', sep='\t', header=0)

# Optional: Strip leading/trailing whitespaces from string columns
manorm_df['Chr'] = manorm_df['Chr'].str.strip()
annotated_df['Chr'] = annotated_df['Chr'].str.strip()

# Adjust 'Start' values if necessary
annotated_df['Start'] = annotated_df['Start'] - 1

# Convert 'Start' and 'End' to integers
manorm_df[['Start', 'End']] = manorm_df[['Start', 'End']].astype(int)
annotated_df[['Start', 'End']] = annotated_df[['Start', 'End']].astype(int)

# Merge DataFrames
merged_df = pd.merge(manorm_df, annotated_df[['Chr', 'Start', 'End', 'Annotation', 'Gene Name', 'Gene Description']], on=['Chr', 'Start', 'End'], how='left')

# Save the merged DataFrame with annotations to a new file
merged_df.to_csv('/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/DiffPeaks/Patient_2920/OPN5vVEH_manorm_annotated.tsv', sep='\t', index=False)

In [45]:
#####################
# Filtering for significantly differentially accessible promoter regions of genes
#####################

# Load the Homer signature file
signatures_df = pd.read_csv('/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/GenomeAnnotation/Tex_OCRs_hg38/Term_Tex_annotated.txt', sep='\t')

# Filter for annotations containing 'promoter'
signatures_promoters_df = signatures_df[signatures_df['Annotation'].str.contains('promoter', case=False)]

# Extract gene names
gene_names_promoter = signatures_promoters_df['Gene Name'].unique()

# Load the fold change file
fold_change_df = pd.read_csv('/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/DiffPeaks/avg_manorm_counts_OPN5vVEH_annot_gene.tsv', sep='\t')

# Convert 'P_value' from scientific notation to float if necessary
fold_change_df['P_value'] = pd.to_numeric(fold_change_df['P_value'], errors='coerce')

# Filter for significantly differentially expressed genes
sig_diff_genes_df = fold_change_df[(fold_change_df['M_value'].abs() > 1) & (fold_change_df['P_value'] < 0.05)]

# Find gene names that are both in the signature promoters list and significantly differentially expressed
overlap_genes = sig_diff_genes_df[sig_diff_genes_df['Gene Name'].isin(gene_names_promoter)]

# Optionally, filter for 'promoter' in the annotation as well
overlap_genes_promoter = overlap_genes[overlap_genes['Annotation'].str.contains('promoter', case=False)]

# Output to a new CSV file
overlap_genes_promoter.to_csv('/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/DiffPeaks/avg_manorm_counts_OPN5vVEH_overlap_genes.tsv', sep='\t', index=False)

# Or simply print the result for further inspection
print(overlap_genes_promoter)

          Chr      Start        End  M_value       P_value  file_count  \
120      chr1    1206496    1206980 -1.09763  1.087271e-13           4   
1173     chr1    7943160    7943618 -2.24342  7.581174e-32           4   
10215    chr1  117001306  117002041 -1.00881  1.419187e-03           4   
16500    chr1  212699064  212700845 -2.13870  5.546527e-56           4   
18457    chr1  239718564  239719501 -1.88983  7.542237e-29           4   
23826   chr10   70602621   70603973 -1.35716  1.527645e-42           4   
39615   chr12    9732932    9733466 -1.08319  1.014907e-05           4   
41953   chr12   50924877   50925031  1.03725  9.872058e-24           1   
49892   chr13   45337621   45337771 -1.95694  5.888096e-41           1   
56993   chr14   88023733   88024229  1.59605  5.516866e-08           4   
77830   chr17   36089576   36090816 -1.08568  5.365882e-39           4   
103781   chr2  113117436  113117986 -1.24289  3.339118e-03           4   
118661  chr21   25572739   25573370 -1

In [2]:
# Load the Excel file
file_path = '/Zulu/timrez/Projects/Wagner_Integrator_HiC_PROseq/INTS1_Regulated_Genelist_forShare.xlsx'  # Change 'your_file.xlsx' to the path of your .xlsx file
genes_of_interest_path = '/Zulu/timrez/Projects/Wagner_Integrator_HiC_PROseq/genes_of_interest.csv'
output_file_path = '/Zulu/timrez/Projects/Wagner_Integrator_HiC_PROseq/INTS1_Regulated_Genelist_forMeta.xlsx'  # Path for the output file

# Read the Excel file into a DataFrame
df = pd.read_excel(file_path, sheet_name='DEseq_INTS1_IAA_plusvsINTS1_IAA')

# # Calculate the difference between 'Stop' and 'Start'
# df['Difference'] = df['Stop'] - df['Start']

# # Filter rows where the difference is greater than 25,000
# filtered_df = df[df['Difference'] > 25000]

# Read the gene names of interest into a list
with open(genes_of_interest_path, 'r') as file:
    genes_of_interest = file.read().splitlines()

# Filter the DataFrame to keep only rows with gene names in the genes of interest list
filtered_df = df[df['geneName'].isin(genes_of_interest)]

# Drop the 'Difference' column as it's no longer needed (optional)
#filtered_df = filtered_df.drop(columns=['Difference'])

# Write the filtered DataFrame to a new Excel file
filtered_df.to_excel(output_file_path, index=False, engine='openpyxl')

print(f"Filtered data written to {output_file_path}")

Filtered data written to /Zulu/timrez/Projects/Wagner_Integrator_HiC_PROseq/INTS1_Regulated_Genelist_forMeta.xlsx


In [3]:
# Step 2: Read the entire Excel file
df = pd.read_excel('/Zulu/timrez/Projects/Wagner_Integrator_HiC_PROseq/INTS1_Regulated_Genelist_forMeta.xlsx')

# Step 4: Save the filtered DataFrame to a new CSV file
df.to_csv('/Zulu/timrez/Projects/Wagner_Integrator_HiC_PROseq/INTS1_Regulated_Genelist_forMeta.csv', index=False, sep="\t")

In [4]:
##########################
# Sorting out peaks of interest
##########################

# Path to your TSV file
file_path = '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/DiffPeaks/avg_manorm_counts_OPN5vVEH_annot_gene.tsv'

# Load the TSV file into a DataFrame
df = pd.read_csv(file_path, sep='\t')

# Step 1: Filter rows where 'file_count' > 3
df_filtered = df[df['file_count'] > 3]

# Step 2: Keep only rows with 'Annotation' containing 'promoter'
df_filtered = df_filtered[df_filtered['Annotation'].str.contains('promoter', case=False, na=False)]

# Step 3: Remove duplicates based on 'Gene_Name' while keeping the row with the largest absolute 'M_value'
df_filtered['abs_M_value'] = df_filtered['M_value'].abs()  # Create a temporary column for absolute M_value
df_filtered = df_filtered.sort_values('abs_M_value', ascending=False).drop_duplicates('Gene_Name').sort_index()
df_filtered = df_filtered.drop('abs_M_value', axis=1)  # Remove the temporary column

# Save the filtered DataFrame to a new TSV file
output_file_path = '/Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/DiffPeaks/avg_manorm_counts_OPN5vVEH_annot_gene_filtered_output.tsv'
df_filtered.to_csv(output_file_path, sep='\t', index=False)

print(f"Filtered data written to {output_file_path}")

Filtered data written to /Zulu/timrez/Projects/ElGamal_D/TCell_ATAC/ENCODE_Pipeline/Data_Analysis/DiffPeaks/avg_manorm_counts_OPN5vVEH_annot_gene_filtered_output.tsv
